In [1]:
!pwd

/home/shankar/1MyProjects/test_poetry_project/notebooks


# When using Google Colab, run the following code

## Mounting Google Drive

In [2]:
# """
# Mounting Google Drive
# """
# from google.colab import drive

# drive.mount("/content/gdrive")

## Install Poetry in the colab runtime environment

In [3]:
# !poetry --version

In [4]:
# !pip install poetry

In [5]:
# !poetry --version

In [6]:
# !poetry config --list

In [7]:
# # Configure poetry to create virtual environments in the project folder

# !poetry config virtualenvs.in-project true

## Installing Project Dependencies

In [8]:
# !pwd

In [9]:
# %cd /content/gdrive/MyDrive/GitHubProjects/test_poetry_project

In [10]:
# !pwd

In [11]:
# # !ls -a
# !rm -rf .venv/

In [12]:
# !make env-colab

* **Adding additional packages to the virtual environment (USEFUL ONLY IF CREATING VIRTUAL ENVIRONMENT IN PROJECT FOLDER IN COLAB)**

In [13]:
# '''
# Add any package like 'torch'
# '''

# !poetry add torch
# !poetry install --no-ansi --no-root

* **Add poetry virtual environment to python path (USEFUL ONLY IF CREATING VIRTUAL ENVIRONMENT IN PROJECT FOLDER IN COLAB)**

In [14]:
# """
# Add poetry virtual environment to python
# path so that all installed dependencies
# can be found by the python interpreter.
# """

# import os
# import sys

# VENV_PATH = "/content/gdrive/MyDrive/GitHubProjects/test_poetry_project/.venv/lib/python3.10/site-packages"
# LOCAL_VENV_PATH = "/content/venv"  # local notebook
# os.symlink(VENV_PATH, LOCAL_VENV_PATH)  # connect to directory in drive
# sys.path.insert(0, LOCAL_VENV_PATH)

* **Use the added packages (EXAMPLE CODE)**

In [15]:
# import torch
# print(torch.__version__)

* **Check the size of the virtual environment**

In [16]:
# !du -hs /content/gdrive/MyDrive/GitHubProjects/test_poetry_project/.venv/

## Format with black

In [17]:
# Install black
# !pip install "black[jupyter]"

# Format using black
# !black /content/gdrive/MyDrive/GitHubProjects/test_poetry_project/notebooks/

# Importing Libraries

In [18]:
# Import basic libraries
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

import calendar
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

# # from nsepython import *
# from bokeh.plotting import figure, show
from nsepythonserver import *

# from bokeh.models import (
#     Arrow,
#     BoxAnnotation,
#     ColumnDataSource,
#     CustomJS,
#     DatetimeTickFormatter,
#     HoverTool,
#     NormalHead,
#     Range1d,
#     Span,
# )
# from bokeh.transform import transform


# from sklearn.model_selection import train_test_split


%matplotlib inline

# Functions

## Technical Indicators

### CalculateRsi

In [19]:
def CalculateRsi(ClosePrices, Period=14):
    """
    Calculates the Relative Strength Index (RSI) for a given series of closing prices.

    Args:
        ClosePrices: A list of close prices for each period.
        Period: The number of periods to consider for the RSI calculation (default: 14).

    Returns:
        A Pandas Series with the calculated RSI values.
    """

    # Calculate price difference
    Delta = ClosePrices.diff()

    # Separate gains and losses
    Gain = Delta.mask(Delta < 0, 0)
    Loss = -Delta.mask(Delta > 0, 0)

    # Calculate average gains and losses over the specified period
    AvgGain = Gain.rolling(window=Period).mean()
    AvgLoss = Loss.rolling(window=Period).mean()

    # Calculate the relative strength (RS)
    RS = AvgGain / AvgLoss

    # Calculate the RSI using the standard formula
    RSI = 100 - (100 / (1 + RS))

    return RSI

### GenerateRsiSignals

In [20]:
def GenerateRsiSignals(RSI, OverboughtThreshold=70, OversoldThreshold=30):
    """
    Generates buy/sell signals based on RSI values.

    Args:
        RSI: A series of RSI values.
        OverboughtThreshold: Threshold for RSI to be considered overbought (default: 70).
        OversoldThreshold: Threshold for RSI to be considered oversold (default: 30).

    Returns:
        A series of signals: 1 for buy, -1 for sell, 0 for no signal.
    """

    signals = [0] * len(RSI)
    for i in range(1, len(RSI)):
        if RSI[i - 1] > OverboughtThreshold and RSI[i] <= OverboughtThreshold:
            signals[i] = -1  # Sell signal when RSI crosses down from overbought
        elif RSI[i - 1] < OversoldThreshold and RSI[i] >= OversoldThreshold:
            signals[i] = 1  # Buy signal when RSI crosses up from oversold

    return signals

### CalculateBollingerBands

In [21]:
def CalculateBollingerBands(ClosePrices, Window=20, NStd=2):
    """
    Calculates the upper and lower Bollinger Bands for a given series of closing prices.

    Args:
        ClosePrices: A list of close prices for each period.
        Window: The window size for the rolling mean and standard deviation (default: 20).
        NStd: The number of standard deviations to use for the band width (default: 2).

    Returns:
        A tuple containing two Pandas Series:
        - UpperBand: The upper Bollinger Band.
        - LowerBand: The lower Bollinger Band.
    """

    # Calculate rolling mean and standard deviation
    RollingMean = ClosePrices.rolling(window=Window).mean()
    RollingStd = ClosePrices.rolling(window=Window).std()

    # Calculate upper and lower Bollinger Bands
    UpperBand = RollingMean + (RollingStd * NStd)
    LowerBand = RollingMean - (RollingStd * NStd)

    return UpperBand, LowerBand

### GenerateBollingerBandSignals

In [22]:
def GenerateBollingerBandSignals(ClosePrices, UpperBand, LowerBand):
    """
    Generates buy/sell signals based on Bollinger Bands.

    Args:
        ClosePrices: A list of close prices for each period.
        UpperBand: A Pandas Series containing the upper Bollinger Band.
        LowerBand: A Pandas Series containing the lower Bollinger Band.

    Returns:
        A Pandas Series with the generated signals (1 for buy, -1 for sell, 0 for no signal).
    """

    signals = [0] * len(ClosePrices)
    for i in range(1, len(ClosePrices)):
        if ClosePrices[i - 1] > UpperBand[i - 1] and ClosePrices[i] <= UpperBand[i]:
            signals[i] = -1  # Sell signal when price crosses down from the upper band
        elif ClosePrices[i - 1] < LowerBand[i - 1] and ClosePrices[i] >= LowerBand[i]:
            signals[i] = 1  # Buy signal when price crosses up from the lower band

    return signals

### CalculateMacd

In [23]:
def CalculateMacd(ClosePrices, NFast=12, NSlow=26):
    """
    Calculates the Moving Average Convergence Divergence (MACD) indicator.

    Args:
        ClosePrices: A list of close prices for each period.
        NFast: The number of periods for the fast exponential moving average (default: 12).
        NSlow: The number of periods for the slow exponential moving average (default: 26).

    Returns:
        A tuple containing three Pandas Series:
        - MACDLine: The MACD line.
        - SignalLine: The signal line.
        - MACDHistogram: The MACD histogram.
    """

    # Calculate exponential moving averages
    EmaFast = ClosePrices.ewm(span=NFast, min_periods=NSlow).mean()
    EmaSlow = ClosePrices.ewm(span=NSlow, min_periods=NSlow).mean()

    # Calculate MACD line
    MACDLine = EmaFast - EmaSlow

    # Calculate signal line
    SignalLine = MACDLine.ewm(span=9, min_periods=9).mean()

    # Calculate MACD histogram
    MACDHistogram = MACDLine - SignalLine

    return MACDLine, SignalLine, MACDHistogram

### GenerateMacdSignals

In [24]:
def GenerateMacdSignals(ClosePrices, MACDLine, SignalLine):
    """
    Generates buy/sell signals based on MACD line crossovers with the signal line.

    Args:
        ClosePrices: A list of close prices for each period.
        MACDLine: A Pandas Series containing the MACD line values.
        SignalLine: A Pandas Series containing the signal line values.

    Returns:
        A Pandas Series with the generated signals (1 for buy, -1 for sell, 0 for no signal).
    """

    signals = [0] * len(ClosePrices)
    for i in range(1, len(ClosePrices)):
        if MACDLine[i - 1] < SignalLine[i - 1] and MACDLine[i] >= SignalLine[i]:
            signals[i] = 1  # Buy signal when MACD line crosses up the signal line
        elif MACDLine[i - 1] > SignalLine[i - 1] and MACDLine[i] <= SignalLine[i]:
            signals[i] = -1  # Sell signal when MACD line crosses down the signal line

    return signals

### CalculateEmas

In [25]:
def CalculateEmas(ClosePrices, Ema1Period, Ema2Period):
    """
    Calculates the Exponential Moving Averages (EMAs) for two given periods.

    Args:
        ClosePrices: A list of close prices for each period.
        Ema1Period: The first EMA period to calculate.
        Ema2Period: The second EMA period to calculate.

    Returns:
        A tuple containing two arrays, Ema1 and Ema2, representing the EMAs for the specified periods.
    """

    # Calculate EMA for Ema1Period
    Alpha1 = 2 / Ema1Period
    Ema1 = [None] * len(ClosePrices)
    for i in range(len(ClosePrices)):
        if i == 0:
            Ema1[i] = ClosePrices[i]
        else:
            Ema1[i] = (ClosePrices[i] * (2 * Alpha1) + Ema1[i - 1] * (1 - Alpha1)) / (
                1 + Alpha1
            )

    # Calculate EMA for Ema2Period
    Alpha2 = 2 / Ema2Period
    Ema2 = [None] * len(ClosePrices)
    for i in range(len(ClosePrices)):
        if i == 0:
            Ema2[i] = ClosePrices[i]
        else:
            Ema2[i] = (ClosePrices[i] * (2 * Alpha2) + Ema2[i - 1] * (1 - Alpha2)) / (
                1 + Alpha2
            )

    return Ema1, Ema2

### GenerateEmaCrossoverSignals

In [26]:
def GenerateEmaCrossoverSignals(ClosePrices, ShortEMA, LongEMA):
    """
    Generates buy/sell signals based on EMA crossovers.

    Args:
        ClosePrices: A list of close prices for each period.
        ShortEMA: A Pandas Series containing the short-term EMA.
        LongEMA: A Pandas Series containing the long-term EMA.

    Returns:
        A Pandas Series with the generated signals (1 for buy, -1 for sell, 0 for no signal).
    """

    signals = [0] * len(ClosePrices)
    for i in range(1, len(ClosePrices)):
        if ShortEMA[i - 1] < LongEMA[i - 1] and ShortEMA[i] >= LongEMA[i]:
            signals[i] = 1  # Buy signal when short EMA crosses above long EMA
        elif ShortEMA[i - 1] > LongEMA[i - 1] and ShortEMA[i] <= LongEMA[i]:
            signals[i] = -1  # Sell signal when short EMA crosses below long EMA

    return signals

### StochasticOscillator

In [27]:
def StochasticOscillator(HighPrices, LowPrices, ClosePrices, KPeriod=14, DPeriod=3):
    """
    Calculates the Stochastic Oscillator (%K and %D) for a given list of high, low, and close prices.

    Args:
        HighPrices: A list of high prices for each period.
        LowPrices: A list of low prices for each period.
        ClosePrices: A list of close prices for each period.
        KPeriod: The period for the %K line (default: 14).
        DPeriod: The period for the %D line (default: 3).

    Returns:
        A tuple containing two arrays: %K and %D.
    """

    k = [0] * len(ClosePrices)
    d = [0] * len(ClosePrices)

    for i in range(KPeriod, len(ClosePrices)):
        highest_high = max(HighPrices[i - KPeriod + 1 : i + 1])
        lowest_low = min(LowPrices[i - KPeriod + 1 : i + 1])
        k[i] = 100 * ((ClosePrices[i] - lowest_low) / (highest_high - lowest_low))

    for i in range(DPeriod, len(ClosePrices)):
        d[i] = sum(k[i - DPeriod + 1 : i + 1]) / DPeriod

    return k, d

### StochasticSignals

In [28]:
def StochasticSignals(ClosePrices, k, d):
    """
    Generates buy/sell signals based on the Stochastic Oscillator.

    Args:
        ClosePrices: A list of close prices for each period.
        k: The %K line of the Stochastic Oscillator.
        d: The %D line of the Stochastic Oscillator.

    Returns:
        A list of signals: 1 for buy, -1 for sell, 0 for no signal.
    """

    signals = [0] * len(ClosePrices)
    for i in range(1, len(ClosePrices)):
        if k[i - 1] < d[i - 1] and k[i] > d[i]:  # Crossover up
            signals[i] = 1  # Buy signal
        elif k[i - 1] > d[i - 1] and k[i] < d[i]:  # Crossover down
            signals[i] = -1  # Sell signal

    return signals

### GenerateTradeSignal

In [29]:
def GenerateTradeSignal(Signal1, Signal2, Signal3, Signal4, Signal5):
    """
    Generates a trade signal based on the aggregate of multiple signals.

    Args:
        Signal1, Signal2, Signal3, Signal4, Signal5: Pandas Series containing signals (1, -1, or 0).

    Returns:
        A Pandas Series with the generated trade signals (1 for buy, -1 for sell, 0 for no signal).
    """

    TradeSignals = []
    for i in range(len(Signal1)):
        AggregateSignal = Signal1[i] + Signal2[i] + Signal3[i] + Signal4[i] + Signal5[i]
        if AggregateSignal > 1:  # More than 30% of signals are 1 (buy)
            TradeSignals.append(1)
        elif AggregateSignal < -1:  # More than 30% of signals are -1 (sell)
            TradeSignals.append(-1)
        else:
            TradeSignals.append(0)  # No clear consensus

    return TradeSignals

### BacktestTradeSignals

In [30]:
def CalculateTradeProfitLoss(Df):
    """
    Calculates the profit or loss for each individual trade signal in a DataFrame,
    records them at their respective entry indices, and marks profitable entries in a 'VerifiedSignal' column.

    Args:
        Df: A pandas DataFrame with columns 'Open', 'High', 'Low', 'Close', and 'TradeSignal'.

    Returns:
        A pandas DataFrame with additional columns 'LongEntry', 'LongExit',
        'ShortEntry', 'ShortExit', 'ProfitLoss', and 'VerifiedSignal'.
    """

    Df["LongEntry"] = 0.0
    Df["LongExit"] = 0.0
    Df["ShortEntry"] = 0.0
    Df["ShortExit"] = 0.0
    Df["ProfitLoss"] = 0.0  # Will now store individual profit/loss values
    Df["VerifiedSignal"] = 0  # Initialize VerifiedSignal column

    OpenPositions = {"Position": [], "EntryPrice": [], "EntryIndex": []}

    for i in range(len(Df)):
        TradeSignal = Df.loc[i, "TradeSignal"]

        if TradeSignal == 1:
            # Close open short positions (if any)
            for j in range(len(OpenPositions["Position"]) - 1, -1, -1):
                if OpenPositions["Position"][j] == "Short":
                    ExitPrice = Df.loc[i + 1, "Open"]  # Use current price as exit price
                    ProfitLoss = (
                        OpenPositions["EntryPrice"][j] - ExitPrice
                    )  # Reversed for short
                    Df.loc[OpenPositions["EntryIndex"][j], "ProfitLoss"] = ProfitLoss
                    Df.loc[OpenPositions["EntryIndex"][j], "ShortExit"] = ExitPrice
                    Df.loc[(OpenPositions["EntryIndex"][j] - 1), "VerifiedSignal"] = (
                        -1 if ProfitLoss > 0 else 0
                    )
                    del OpenPositions["Position"][j]
                    del OpenPositions["EntryPrice"][j]
                    del OpenPositions["EntryIndex"][j]

            # Open a long position
            OpenPositions["Position"].append("Long")
            OpenPositions["EntryPrice"].append(Df.loc[i + 1, "Open"])
            OpenPositions["EntryIndex"].append(i + 1)
            Df.loc[i + 1, "LongEntry"] = Df.loc[i + 1, "Open"]

        elif TradeSignal == -1:
            # Close open long positions (if any)
            for j in range(len(OpenPositions["Position"]) - 1, -1, -1):
                if OpenPositions["Position"][j] == "Long":
                    ExitPrice = Df.loc[i + 1, "Open"]
                    ProfitLoss = ExitPrice - OpenPositions["EntryPrice"][j]
                    Df.loc[OpenPositions["EntryIndex"][j], "ProfitLoss"] = ProfitLoss
                    Df.loc[OpenPositions["EntryIndex"][j], "LongExit"] = ExitPrice
                    Df.loc[(OpenPositions["EntryIndex"][j] - 1), "VerifiedSignal"] = (
                        1 if ProfitLoss > 0 else 0
                    )
                    del OpenPositions["Position"][j]
                    del OpenPositions["EntryPrice"][j]
                    del OpenPositions["EntryIndex"][j]

            # Open a short position
            OpenPositions["Position"].append("Short")
            OpenPositions["EntryPrice"].append(Df.loc[i + 1, "Open"])
            OpenPositions["EntryIndex"].append(i + 1)
            Df.loc[i + 1, "ShortEntry"] = Df.loc[i + 1, "Open"]

    return Df

# Variables

# Data Import, Cleaning

## Data Import from Google Colab

In [31]:
# '''
# Upload the file from local system to Google Colab
# '''
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# """
# Read the file from Google Drive
# """
# RawNiftyDf = pd.read_csv(
#     "/content/gdrive/MyDrive/GitHubProjects/test_poetry_project/nifty_historic.csv"
# )
# RawIndiaVixDf = pd.read_csv(
#     "/content/gdrive/MyDrive/GitHubProjects/test_poetry_project/indiavix_historic.csv"
# )

## Data Import from Asmsl

In [32]:
# """
# Read the file from Asmsl
# """
# RawNiftyDf = pd.read_csv(
#     "/home/studio-lab-user/1MyProjects/test_poetry_project/data/raw/nifty_historic.csv"
# )
# RawIndiaVixDf = pd.read_csv(
#     "/home/studio-lab-user/1MyProjects/test_poetry_project/data/raw/indiavix_historic.csv"
# )

## Data Import from Desktop

In [33]:
"""
Read the file from Desktop
"""
RawNiftyDf = pd.read_excel(
    "/home/shankar/1MyProjects/test_poetry_project/data/raw/RawNiftyHistoricData.xlsx"
)

RawIndiaVixDf = pd.read_excel(
    "/home/shankar/1MyProjects/test_poetry_project/data/raw/RawIndiaVixHistoricData.xlsx"
)

## Create NiftyDf as a copy of RawNiftyDf

In [34]:
NiftyDf = RawNiftyDf.copy()

In [35]:
NiftyDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   EOD_INDEX_NAME       1977 non-null   object 
 1   EOD_OPEN_INDEX_VAL   1977 non-null   float64
 2   EOD_HIGH_INDEX_VAL   1977 non-null   float64
 3   EOD_CLOSE_INDEX_VAL  1977 non-null   float64
 4   EOD_LOW_INDEX_VAL    1977 non-null   float64
 5   EOD_TIMESTAMP        1977 non-null   object 
dtypes: float64(4), object(2)
memory usage: 92.8+ KB


In [36]:
# Delete the columns 'Index Name' and 'INDEX_NAME'
NiftyDf.drop(["EOD_INDEX_NAME"], axis=1, inplace=True)

In [37]:
# Rename all column names
NiftyDf.rename(
    columns={
        "EOD_OPEN_INDEX_VAL": "Open",
        "EOD_HIGH_INDEX_VAL": "High",
        "EOD_CLOSE_INDEX_VAL": "Close",
        "EOD_LOW_INDEX_VAL": "Low",
        "EOD_TIMESTAMP": "Date",
    },
    inplace=True,
)

# Convert date column to datetime object
NiftyDf["Date"] = pd.to_datetime(
    pd.to_datetime(NiftyDf["Date"], format="%d-%b-%Y").dt.strftime("%Y-%m-%d 15:30:00")
)

# Convert the variables 'open', 'high', 'low', 'close' to float
NiftyDf["Open"] = NiftyDf["Open"].astype(float)
NiftyDf["High"] = NiftyDf["High"].astype(float)
NiftyDf["Low"] = NiftyDf["Low"].astype(float)
NiftyDf["Close"] = NiftyDf["Close"].astype(float)

NiftyDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Open    1977 non-null   float64       
 1   High    1977 non-null   float64       
 2   Close   1977 non-null   float64       
 3   Low     1977 non-null   float64       
 4   Date    1977 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4)
memory usage: 77.4 KB


In [38]:
NiftyDf[NiftyDf.duplicated()]

,Open,High,Close,Low,Date
1775,17680.35,17799.95,17711.45,17671.95,2023-03-06 15:30:00


In [39]:
NiftyDf.drop_duplicates(inplace=True)

In [40]:
# Sort the NiftyDf by Date column in ascending order and sort the index
NiftyDf.sort_values(by="Date", ascending=True, inplace=True)
NiftyDf.reset_index(drop=True, inplace=True)

In [41]:
# Create a new variables called dayofweek such as Monday, Tuesday, etc. and numdayofweek
# NiftyDf["DayOfWeek"] = NiftyDf["Date"].dt.day_name()
# NiftyDf["NumDayOfWeek"] = NiftyDf["Date"].dt.dayofweek

In [42]:
# # Calculate unique week number with week ending on Thursday

# MinDate = NiftyDf["Date"].min()
# NiftyDf["WeekNumber"] = ((NiftyDf["Date"] - MinDate).dt.days // 7).astype(int)
# NiftyDf["WeekNumber"] += NiftyDf["Date"].dt.dayofweek > 3

In [43]:
# Create a new empty column named 'ExpiryDay'
# NiftyDf["ExpiryDay"] = None

In [44]:
# # Create a variable named StartDate with values 2000-01-01 09:15:00 as a timestamp
# StartDate = pd.to_datetime("2000-01-01 15:30:00")
# print("StartDate : ", StartDate)
# print("---------------------------")

# # Create a variable named MonthsCount from NiftyDf['Date']
# MonthsCount = NiftyDf["Date"].dt.to_period("M").nunique()
# print("MonthsCount : ", MonthsCount)
# print("---------------------------")

In [45]:
NiftyDf

,Open,High,Close,Low,Date
0,7938.45,7972.55,7963.20,7909.80,2016-01-01 15:30:00
1,7924.55,7937.55,7791.30,7781.10,2016-01-04 15:30:00
2,7828.40,7831.20,7784.65,7763.25,2016-01-05 15:30:00
3,7788.05,7800.95,7741.00,7721.20,2016-01-06 15:30:00
4,7673.35,7674.95,7568.30,7556.60,2016-01-07 15:30:00
...,...,...,...,...,...
1971,21295.85,21390.50,21349.40,21232.45,2023-12-22 15:30:00
1972,21365.20,21477.15,21441.35,21329.45,2023-12-26 15:30:00
1973,21497.65,21675.75,21654.75,21495.80,2023-12-27 15:30:00
1974,21715.00,21801.45,21778.70,21678.00,2023-12-28 15:30:00


# Copy RawIndiaVixDf to IndiaVixDf and Clean

In [46]:
IndiaVixDf = RawIndiaVixDf.copy()
IndiaVixDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   1979 non-null   object 
 1   EOD_TIMESTAMP         1979 non-null   object 
 2   EOD_INDEX_NAME        1979 non-null   object 
 3   EOD_OPEN_INDEX_VAL    1979 non-null   float64
 4   EOD_CLOSE_INDEX_VAL   1979 non-null   float64
 5   EOD_HIGH_INDEX_VAL    1979 non-null   float64
 6   EOD_LOW_INDEX_VAL     1979 non-null   float64
 7   EOD_PREV_CLOSE        1979 non-null   float64
 8   TIMESTAMP             1979 non-null   object 
 9   createdAt             1979 non-null   object 
 10  updatedAt             1979 non-null   object 
 11  __v                   1979 non-null   int64  
 12  VIX_PERC_CHG          1979 non-null   float64
 13  VIX_PTS_CHG           1979 non-null   float64
 14  ALTERNATE_INDEX_NAME  1979 non-null   object 
dtypes: float64(7), int64(

In [47]:
# Drop the columns from IndiaVixDf : _id, EOD_INDEX_NAME, TIMESTAMP, ALTERNATE_INDEX_NAME, createdAt, updatedAt, __v
IndiaVixDf = IndiaVixDf.drop(
    columns=[
        "_id",
        "EOD_INDEX_NAME",
        "TIMESTAMP",
        "createdAt",
        "updatedAt",
        "__v",
        "ALTERNATE_INDEX_NAME",
    ]
)

In [48]:
# Rename columns of IndiaVixDf : Date, Open, High, Low, Close, Volume
IndiaVixDf = IndiaVixDf.rename(
    columns={
        "EOD_TIMESTAMP": "Date",
        "EOD_OPEN_INDEX_VAL": "Open",
        "EOD_CLOSE_INDEX_VAL": "Close",
        "EOD_HIGH_INDEX_VAL": "High",
        "EOD_LOW_INDEX_VAL": "Low",
        "EOD_PREV_CLOSE": "PrevClose",
        "VIX_PTS_CHG": "PtsChange",
        "VIX_PERC_CHG": "PctChange",
    }
)
IndiaVixDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1979 non-null   object 
 1   Open       1979 non-null   float64
 2   Close      1979 non-null   float64
 3   High       1979 non-null   float64
 4   Low        1979 non-null   float64
 5   PrevClose  1979 non-null   float64
 6   PctChange  1979 non-null   float64
 7   PtsChange  1979 non-null   float64
dtypes: float64(7), object(1)
memory usage: 123.8+ KB


In [49]:
# Convert TimeStamp to DateTime in IndiaVixDf
IndiaVixDf["Date"] = pd.to_datetime(
    pd.to_datetime(IndiaVixDf["Date"], format="%d-%b-%Y").dt.strftime(
        "%Y-%m-%d 15:30:00"
    )
)
IndiaVixDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1979 non-null   datetime64[ns]
 1   Open       1979 non-null   float64       
 2   Close      1979 non-null   float64       
 3   High       1979 non-null   float64       
 4   Low        1979 non-null   float64       
 5   PrevClose  1979 non-null   float64       
 6   PctChange  1979 non-null   float64       
 7   PtsChange  1979 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 123.8 KB


In [50]:
# Sort IndiaVixDf by TimeStamp and reset index
IndiaVixDf = IndiaVixDf.sort_values(by="Date")
IndiaVixDf = IndiaVixDf.reset_index(drop=True)

In [51]:
IndiaVixDf[IndiaVixDf.duplicated()]

,Date,Open,Close,High,Low,PrevClose,PctChange,PtsChange
1884,2023-08-16 15:30:00,11.9950,12.1250,12.6825,11.9950,11.9950,1.083785,0.1300
1930,2023-10-20 15:30:00,10.8975,10.8175,11.0900,10.0750,10.8975,-0.734113,-0.0800
1973,2023-12-21 15:30:00,14.4500,13.7575,15.0100,12.2825,14.4500,-4.792388,-0.6925


In [52]:
IndiaVixDf.drop_duplicates(inplace=True)

In [53]:
IndiaVixDf

,Date,Open,Close,High,Low,PrevClose,PctChange,PtsChange
0,2016-01-01 15:30:00,13.8700,14.2600,14.3475,13.1050,13.8700,2.811824,0.3900
1,2016-01-04 15:30:00,14.2600,16.8350,17.1025,14.0300,14.2600,18.057504,2.5750
2,2016-01-05 15:30:00,16.8350,16.7025,16.8350,15.5675,16.8350,-0.787051,-0.1325
3,2016-01-06 15:30:00,16.7025,16.5525,17.1625,15.6075,16.7025,-0.898069,-0.1500
4,2016-01-07 15:30:00,16.5525,18.9600,19.2550,16.4900,16.5525,14.544631,2.4075
...,...,...,...,...,...,...,...,...
1974,2023-12-22 15:30:00,13.7575,13.7075,13.9875,12.2250,13.7575,-0.363438,-0.0500
1975,2023-12-26 15:30:00,13.7075,14.6775,14.9500,13.7075,13.7075,7.076418,0.9700
1976,2023-12-27 15:30:00,14.6775,15.5575,15.7325,14.4700,14.6775,5.995571,0.8800
1977,2023-12-28 15:30:00,15.5575,15.1400,16.4700,14.9625,15.5575,-2.683593,-0.4175


# Impute IndiaVix values in NiftyDf

In [54]:
NiftyDf["IndiaVix"] = None

In [55]:
for Date in IndiaVixDf["Date"].values:
    if Date in NiftyDf["Date"].values:
        # print(f"{Date} is in NiftyDf:", True)
        # print("---------------------------")
        NiftyDf.loc[NiftyDf["Date"] == Date, "IndiaVix"] = IndiaVixDf.loc[
            IndiaVixDf["Date"] == Date, "Close"
        ].values[0]
    else:
        # print(f"{Date} is NOT in NiftyDf:", False)
        # print("---------------------------")
        continue

In [56]:
# Show rows that have IndiaVix = NaN
NiftyDf.isnull().sum()

Open        0
High        0
Close       0
Low         0
Date        0
IndiaVix    0
dtype: int64

In [57]:
# # Impute IndiaVix values from Tradingview on the missing dates

# NiftyDf.loc[NiftyDf["Date"] == "2020-07-15 15:30:00", "IndiaVix"] = 26.2900
# NiftyDf.loc[NiftyDf["Date"] == "2020-07-16 15:30:00", "IndiaVix"] = 25.3625
# NiftyDf.loc[NiftyDf["Date"] == "2020-07-17 15:30:00", "IndiaVix"] = 24.1575
# NiftyDf.loc[NiftyDf["Date"] == "2020-07-20 15:30:00", "IndiaVix"] = 24.7125

In [58]:
# # Drop rows that have IndiaVix = NaN and reset index
# NiftyDf = NiftyDf[NiftyDf["IndiaVix"].notnull()]
# NiftyDf = NiftyDf.reset_index(drop=True)

In [59]:
NiftyDf["IndiaVix"] = NiftyDf["IndiaVix"].astype(float)

# Generate Indicator Signals

In [60]:
# CalulateRsi
NiftyDf["Rsi"] = CalculateRsi(NiftyDf["Close"], 14)

# GenerateRsiSignals
NiftyDf["RsiSignal"] = GenerateRsiSignals(
    NiftyDf["Rsi"], OverboughtThreshold=70, OversoldThreshold=30
)

# CalulateBollingerBands
NiftyDf["BbUpper"], NiftyDf["BbLower"] = CalculateBollingerBands(
    NiftyDf["Close"], Window=20, NStd=2
)

# GenerateBollingerBandSignals
NiftyDf["BbSignal"] = GenerateBollingerBandSignals(
    NiftyDf["Close"], NiftyDf["BbUpper"], NiftyDf["BbLower"]
)

# CalculateMacd
NiftyDf["MacdLine"], NiftyDf["SignalLine"], NiftyDf["MacdHistogram"] = CalculateMacd(
    NiftyDf["Close"], NFast=12, NSlow=26
)

# GenerateMacdSignals
NiftyDf["MacdSignal"] = GenerateMacdSignals(
    NiftyDf["Close"], NiftyDf["MacdLine"], NiftyDf["SignalLine"]
)

# CalculateEmas
NiftyDf["Ema1"], NiftyDf["Ema2"] = CalculateEmas(
    NiftyDf["Close"], Ema1Period=20, Ema2Period=50
)

# GenerateEmaSignals
NiftyDf["EmasSignal"] = GenerateEmaCrossoverSignals(
    NiftyDf["Close"], NiftyDf["Ema1"], NiftyDf["Ema2"]
)

# StochasticOscillator
NiftyDf["StochK"], NiftyDf["StochD"] = StochasticOscillator(
    NiftyDf["High"], NiftyDf["Low"], NiftyDf["Close"], KPeriod=14, DPeriod=3
)

# StochasticSignals
NiftyDf["StochasticSignal"] = StochasticSignals(
    NiftyDf["Close"], NiftyDf["StochK"], NiftyDf["StochD"]
)

## GenerateTradeSignals

In [61]:
# GenerateTradeSignals

NiftyDf["TradeSignal"] = GenerateTradeSignal(
    NiftyDf["RsiSignal"],
    NiftyDf["BbSignal"],
    NiftyDf["MacdSignal"],
    NiftyDf["EmasSignal"],
    NiftyDf["StochasticSignal"],
)

## TradeSignal Value Count

In [62]:
NiftyDf["TradeSignal"].value_counts()

TradeSignal
 0    1827
-1      81
 1      68
Name: count, dtype: int64

## BacktestTradeSignals

In [63]:
NiftyDf = CalculateTradeProfitLoss(NiftyDf)

In [64]:
print("Strategy Performance :", NiftyDf["ProfitLoss"].sum())
print("Strategy Sharpe Ratio :", NiftyDf["ProfitLoss"].mean() / NiftyDf["ProfitLoss"].std())
print("Strategy Max Drawdown :", NiftyDf["ProfitLoss"].min())
print("---------------------------")
print("Buy and Hold Performance :", (NiftyDf.iloc[-1]["Close"] - NiftyDf.iloc[0][ "Open"]))


Strategy Performance : 7299.3500000000095
Strategy Sharpe Ratio : 0.02342762908892103
Strategy Max Drawdown : -1921.9500000000007
---------------------------
Buy and Hold Performance : 13792.95


In [65]:
NiftyDf["VerifiedSignal"].value_counts()

VerifiedSignal
 0    1893
 1      48
-1      35
Name: count, dtype: int64

In [66]:
NiftyDf.query("TradeSignal != 0 and TradeSignal != VerifiedSignal")["TradeSignal"].value_counts()

TradeSignal
-1    46
 1    20
Name: count, dtype: int64

In [67]:
# Assuming your DataFrame is named 'NiftyDf'

fig = go.Figure(
    go.Candlestick(
        x=NiftyDf["Date"],
        open=NiftyDf["Open"],
        high=NiftyDf["High"],
        low=NiftyDf["Low"],
        close=NiftyDf["Close"],
        increasing_line_color="green",
        decreasing_line_color="red",
    )
)

# Add ProfitLoss line plot
fig.add_trace(
    go.Scatter(
        x=NiftyDf["Date"],
        y=NiftyDf["ProfitLoss"],
        mode="lines",
        line=dict(color="blue", width=1),
        name="ProfitLoss",
    )
)

# Add Cumulative ProfitLoss line
fig.add_trace(
    go.Scatter(
        x=NiftyDf["Date"],
        y=NiftyDf["ProfitLoss"].cumsum(),
        mode="lines",
        line=dict(color="teal", width=2),
        name="Cumulative ProfitLoss",
    )
)

# Add green arrows for verified signals
fig.add_trace(
    go.Scatter(
        x=NiftyDf[NiftyDf["VerifiedSignal"] == True]["Date"],
        y=NiftyDf[NiftyDf["VerifiedSignal"] == True]["High"],
        mode="markers",
        marker=dict(symbol="triangle-up", color="green", size=15),
        name="Verified Signals",
    )
)

# Add red arrows for non-verified signals
fig.add_trace(
    go.Scatter(
        x=NiftyDf.query("TradeSignal != 0 and TradeSignal != VerifiedSignal")["Date"],
        y=NiftyDf.query("TradeSignal != 0 and TradeSignal != VerifiedSignal")["High"],
        mode="markers",
        marker=dict(symbol="triangle-down", color="red", size=15),
        name="Non-Verified Signals",
    )
)

# Customize the layout
fig.update_layout(
    title="Candlestick Chart with Signals",
    xaxis_title="Date",
    yaxis_title="Price",
)

pio.write_html(
    fig,
    file="/home/shankar/1MyProjects/test_poetry_project/plots/NiftyBacktestingTradeSignals.html",
)

fig.show(renderer="browser")

Opening in existing browser session.


In [68]:
NiftyDf[20:80]

,Open,High,Close,Low,Date,IndiaVix,Rsi,RsiSignal,BbUpper,BbLower,BbSignal,MacdLine,SignalLine,MacdHistogram,MacdSignal,Ema1,Ema2,EmasSignal,StochK,StochD,StochasticSignal,TradeSignal,LongEntry,LongExit,ShortEntry,ShortExit,ProfitLoss,VerifiedSignal
20,7589.50,7600.45,7555.95,7541.25,2016-02-01 15:30:00,17.8900,49.551136,0,7800.881027,7230.163973,0,NaN,NaN,NaN,0,7481.245847,7580.164647,0,86.553812,73.925651,0,0,0.00,0.00,0.00,0.00,0.0,0
21,7566.65,7576.30,7455.55,7428.05,2016-02-02 15:30:00,18.0350,47.046448,0,7753.664150,7243.805850,0,NaN,NaN,NaN,0,7476.573875,7570.578905,0,58.918249,78.014890,-1,0,0.00,0.00,0.00,0.00,0.0,0
22,7392.45,7419.40,7361.80,7350.30,2016-02-03 15:30:00,18.5875,39.643779,0,7700.851067,7254.333933,0,NaN,NaN,NaN,0,7455.705898,7554.518989,0,33.113130,59.528397,0,0,0.00,0.00,0.00,0.00,0.0,0
23,7411.45,7457.05,7404.00,7365.95,2016-02-04 15:30:00,18.2075,43.259568,0,7648.311287,7273.173713,0,NaN,NaN,NaN,0,7446.304825,7542.940605,0,45.270929,45.767436,0,0,0.00,0.00,0.00,0.00,0.0,0
24,7418.25,7503.15,7489.10,7406.65,2016-02-05 15:30:00,18.1150,52.641063,0,7638.027688,7275.537312,0,NaN,NaN,NaN,0,7454.085766,7538.799020,0,68.978966,49.121008,1,0,0.00,0.00,0.00,0.00,0.0,0
25,7489.70,7512.55,7387.25,7363.20,2016-02-08 15:30:00,20.2575,51.837769,0,7616.327905,7275.827095,0,-23.412065,NaN,NaN,0,7441.933809,7527.141403,0,40.604541,51.618146,-1,0,0.00,0.00,0.00,0.00,0.0,0
26,7303.95,7323.45,7298.20,7275.15,2016-02-09 15:30:00,21.7275,43.094229,0,7605.786390,7259.803610,0,-31.683008,NaN,NaN,0,7415.800389,7509.530526,0,15.796072,41.793193,0,0,0.00,0.00,0.00,0.00,0.0,0
27,7264.30,7271.85,7215.70,7177.75,2016-02-10 15:30:00,22.4650,45.062770,0,7612.152531,7223.977469,0,-43.661967,NaN,NaN,0,7379.418500,7486.928178,0,8.977999,21.792870,0,0,0.00,0.00,0.00,0.00,0.0,0
28,7203.60,7208.65,6976.35,6959.95,2016-02-11 15:30:00,25.9650,36.990691,0,7654.741990,7122.783010,0,-69.754502,NaN,NaN,0,7306.133318,7447.652933,0,2.560500,9.111523,0,0,0.00,0.00,0.00,0.00,0.0,0
29,7023.65,7034.80,6980.95,6869.00,2016-02-12 15:30:00,24.6725,28.223340,0,7673.846854,7048.093146,0,-88.980261,NaN,NaN,0,7247.009079,7411.752708,0,15.305216,8.947905,1,0,0.00,0.00,0.00,0.00,0.0,0


In [69]:
NiftyDf.to_excel(
    "/home/shankar/1MyProjects/test_poetry_project/data/processed/NiftyDfVerifiedTradeSignal.xlsx",
    index=False,
)